In [1]:
; cl-waffeの準備
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)

T

(:CL-WAFFE)

T

Unable to find framework CUDA


## WaffeTensorの基本的な使い方

WaffeTensorには三種類あります

- const 定数として扱われるWaffeTensor (e.g.: データセット)
- tensor 勾配を必要とするWaffeTensor (e.g.: モデルの重み) これはTensorの行列と、勾配の行列を保存するから、固定のメモリ使用量が単純に**二倍**になる。
- sysconst 計算途中で大量に作成するTensor, 一部型判定などを省略。 (e.g.: !add関数の帰り値)

In [6]:
(const 0)

#Const(0)

In [7]:
(tensor 0)

#Parameter{0}

In [8]:
(sysconst 0)

#Const(0)

## 計算ノードの表示

テンソルの:backawrdスロットに、逆伝播を計算するためのNodeが記録されていく。

In [9]:
(defparameter *a* (!randn `(10 10)))
(!add *a* 1.0)

*A*

#Const(((1.625... -0.29... ~ 0.376... 0.299...)        
                 ...
        (-0.04... 1.507... ~ 0.486... 0.417...)) :mgl t :shape (10 10) :backward <Node: ADDTENSOR{W937}>)

## Transpose/Tracing JITによる遅延評価

以下の機能はTensorを遅延評価することで実現される

1. `!transpose`関数(演算を遅延評価して、最後の二つの軸を入れ替える)
2. Tracing JIT (Waffe標準演算を遅延評価して、(value tensor)が呼ばれたら計算ノードを探索+動的にカーネルを定義)

遅延評価されたTensorはλ式として表現されるが、shapeにはアクセスできる。

`!reshape`などの関数も普通に使える（特別なコードを加える必要はない）が、その場合遅延評価が受理されるので注意

Tracing JITは`with-jit`マクロ内部で有効, `cl-waffe::with-jit-debug`に書き換えると、内部で何をしてるのか分かり易いと思う。

In [19]:
(defparameter *a* (!randn `(3 3)))

(with-jit
    (let ((out (!exp (!log *a*))))
         (print (!shape out)) ;shapeにアクセス
         out)) ; outは遅延評価されている

*A*

#Const(#<FUNCTION (LABELS CL-WAFFE.BACKENDS.MGL::LAZYEVALUATEDNODES :IN CL-WAFFE.BACKENDS.MGL::STEP-AND-PRODUCE-LAZY-EVAL) {100178B48B}>)


(3 3) 

In [26]:
(with-jit
    (let ((out (!exp (!log *a*))))
         (print (value out)) ; valueで遅延評価を受理, valueはmgl-mat:matを返す
         out))

#Const(((2.242... 0.252... 1.908...)        
                 ...
        (2.242... 0.355... 1.056...)) :mgl t :shape (3 3) :backward <Node: EXPTENSOR{W1043}>)


#<MGL-MAT:MAT 3x3 B #2A((2.2420775e-44 0.25249693 1.9089459)
                        (2.2420775e-44 0.401076 2.2420775e-44)
                        (2.2420775e-44 0.3555825 1.0564866))> 

In [28]:
(defparameter *m* (!randn `(2 3 2)))
(defparameter *n* (!randn `(3 2)))

(!transpose *m*)

*M*

*N*

#Const(#<FUNCTION (LABELS CL-WAFFE.BACKENDS.MGL::LAZYTRANSPOSE :IN CL-WAFFE.BACKENDS.MGL::LAZY-EVAL-TRANSPOSE) {1002C531FB}>)

In [29]:
(!shape *)

(2 2 3)

In [32]:
(time (!matmul (!transpose *m*) *n*)) ; matmulの前のTransposeは遅延評価をしておけば余計なコストがかからない！

#Const((((3.518... -2.21...)
         (0.137... 0.245...))
        ((1.567... -0.18...)
         (0.946... -1.62...))) :mgl t :shape (2 2 2) :backward <Node: MATMULTENSOR{W1084}>)

Evaluation took:
  0.000 seconds of real time
  0.000063 seconds of total run time (0.000063 user, 0.000000 system)
  100.00% CPU
  143,348 processor cycles
  0 bytes consed
  


## Shapeにアクセス

Shapeにアクセスする関数など